# <center> 模拟前端

In [ ]:
# ========== 完整流程：上传文件 -> 通过文件名查询 ==========

import requests
import json
import os

rag_base_url = "http://localhost:5000"  # RAG服务地址 (记得先启动redis!)
base_url = "http://localhost:8000"      # ChatAgent服务地址
file_path = "./test.pdf"

# ========== 步骤1: 上传文件到RAG服务，获取文件信息 ==========
def upload_file_and_get_info(rag_service_base_url: str, file_path: str, extract_images: bool = False):
    try:
        upload_url = f"{rag_service_base_url}/upload-pdf?extract_images={extract_images}"
        with open(file_path, "rb") as f:
            upload_response = requests.post(upload_url, files={"file": f})
        
        if upload_response.status_code != 200:
            print(f"上传失败，状态码: {upload_response.status_code}")
            print(f"错误信息: {upload_response.text}")
            return None
        
        data = upload_response.json()["data"]
        return {
            "filename": data["filename"],  # 文件名
            "kb_id": data["knowledge_base_id"],  # 知识库ID
            "file_size": data["file_size"],
            "chunk_count": data["chunk_count"],
            "token_stats": data["token_stats"]
        }
    except Exception as e:
        print(f"上传文件错误：{e}")
        return None

# 上传文件并获取信息
print("=" * 60)
print("步骤1: 上传文件到RAG服务...")
print("=" * 60)

file_info = upload_file_and_get_info(
    rag_service_base_url=rag_base_url, 
    file_path=file_path, 
    extract_images=False
)

if not file_info:
    print("❌ 文件上传失败，请检查文件路径和RAG服务状态")
else:
    print("✅ 文件上传成功！")
    print(f"   文件名: {file_info['filename']}")
    print(f"   知识库ID: {file_info['kb_id']}")
    print(f"   文件大小: {file_info['file_size']} bytes")
    print(f"   分块数量: {file_info['chunk_count']}")
    print(f"   Token统计: {file_info['token_stats']}")
    
    # ========== 步骤2: 使用文件名进行查询 ==========
    print("\n" + "=" * 60)
    print("步骤2: 通过文件名查询知识库...")
    print("=" * 60)
    
    url = "http://localhost:8000/v1/chat/completions"
    FIXED_SESSION_ID = "my_test_session_filename_query"
    
    # 在查询中直接使用文件名，Agent会自动通过文件名找到对应的知识库
    request_data = {
        "user_id": "test_user3",
        "session_id": FIXED_SESSION_ID,
        "query": f"请查询文件名为'{file_info['filename']}'的知识库中关于Transformer的内容"
    }
    
    print(f"查询请求: {request_data['query']}\n")
    
    response = requests.post(
        url,
        json=request_data,
        stream=True,
        headers={"Content-Type": "application/json"}
    )
    
    for line in response.iter_lines():
        if line:
            line_str = line.decode("utf-8").strip()
            if line_str.startswith("data: "):
                json_str = line_str[6:]
                try:
                    resp = json.loads(json_str)
                    
                    # 1. AI逐字符流式输出: 前端要实时渲染为markdown
                    if resp["type"] == "content":
                        print(resp["content"], end="", flush=True)
                    
                    # 2. AI正在使用工具：需要标记为特殊类型
                    elif resp["type"] == "tool_call":
                        print(f"\n\n[工具调用]")
                        print(f"  工具名称: {resp['content']['tool_name']}")
                        print(f"  调用参数: {resp['content']['tool_args']}")
                    
                    # 3. AI工具返回结果: 需要溯源验证
                    elif resp["type"] == "tool_result":
                        try:
                            tool_result = json.loads(resp["content"]["called_tool_content"])
                            print(f"\n[工具返回结果]")
                            print(f"  状态码: {tool_result['metadata'].get('status_code', 'N/A')}")
                            print(f"  知识库ID: {tool_result['metadata'].get('kb_id', 'N/A')}")
                            print(f"  文件名: {tool_result['metadata'].get('filename', 'N/A')}")
                            print(f"  内容预览: {tool_result['content'][:200]}...")
                        except:
                            print(f"\n[工具返回结果] {resp['content']}")
                    
                    # 4. 单轮对话token消耗: 需要统计当前用户点数
                    elif resp["type"] == "usage":
                        print(f"\n\n[Token使用统计]")
                        print(f"  输入Token: {resp['content']['input_tokens']}")
                        print(f"  输出Token: {resp['content']['output_tokens']}")
                        print(f"  总计Token: {resp['content']['total_tokens']}")
                except Exception as e:
                    pass


步骤1: 上传文件到RAG服务...
✅ 文件上传成功！
   文件名: test.pdf
   知识库ID: 2fb820d3-5cb9-46d5-9126-fa5b7412c36c
   文件大小: 2228535 bytes
   分块数量: 44
   Token统计: {'vectorbase_usage': 7864, 'query_usage': 0, 'total_usage': 7864}

步骤2: 通过文件名查询知识库...
查询请求: 请查询文件名为'test.pdf'的知识库中关于Transformer的内容

我将帮您查询文件名为'test.pdf'的知识库中关于Transformer的内容。

[工具调用]
  工具名称: semantic_search
  调用参数: {}

[工具返回结果]
  状态码: 200
  知识库ID: 2fb820d3-5cb9-46d5-9126-fa5b7412c36c
  文件名: test.pdf
  内容预览: Transformer
基本背景
 Transformer由论文《Attention is All You Need》提出，现在是谷歌云TPU推荐的参考模型。论文相关的
Tensorflow的代码可以从GitHub获取，其作为Tensor2Tensor包的一部分。哈佛的NLP团队也实现了一个基于
PyTorch的版本，并注释该论文。
 Transformer解决了RNN无法并行计算， 长距离信息遗...
根据查询结果，在'test.pdf'文件中关于Transformer的内容如下：

## Transformer概述

Transformer是由论文《Attention is All You Need》提出的模型，现在是谷歌云TPU推荐的参考模型。该模型解决了传统RNN的两个主要问题：
1. **无法并行计算**
2. **长距离信息遗忘**

## 核心特点
Transformer彻底摒弃了RNN和CNN结构，转而使用**注意力机制**来解决问题。

## 整体结构
Transformer采用**Encoder-Decoder结构**：

### Encoder（编码器）
负责输入序列特征编码，包含以下处理步骤：
1. **前置处理**：
   - 词嵌入（token-embedd

In [ ]:
# ========== 方式2: 先获取文件名对应的kb_id，然后查询 ==========
# 如果你已经上传了文件，可以通过文件名获取kb_id

def get_kb_id_by_filename(rag_service_base_url: str, filename: str):
    """通过文件名获取知识库ID"""
    try:
        get_kb_id_url = f"{rag_service_base_url}/get-kb-id-by-filename"
        response = requests.get(
            get_kb_id_url,
            params={"filename": filename},
            timeout=30
        )
        if response.status_code == 200:
            return response.json()["data"]["knowledge_base_id"]
        else:
            print(f"错误：{response.text}")
            return None
    except Exception as e:
        print(f"错误：{e}")
        return None

# 示例：通过文件名获取kb_id（前提是文件已经上传）
filename = "test.pdf"
print(f"正在查找文件名 '{filename}' 对应的知识库ID...")

kb_id= get_kb_id_by_filename(rag_base_url, filename)

if kb_id:
    print(f"✅ 找到知识库ID: {kb_id}")
    
    # 然后可以使用kb_id进行查询
    url = "http://localhost:8000/v1/chat/completions"
    request_data = {
        "user_id": "test_user3",
        "session_id": "my_test_session_kb_id_query",
        "query": f"请查询知识库{kb_id}中关于Transformer的内容"
    }
    
    print(f"\n使用获取到的kb_id进行查询...")
    print(f"查询请求: {request_data['query']}\n")
    
    response = requests.post(
        url,
        json=request_data,
        stream=True,
        headers={"Content-Type": "application/json"}
    )
    
    for line in response.iter_lines():
        if line:
            line_str = line.decode("utf-8").strip()
            if line_str.startswith("data: "):
                json_str = line_str[6:]
                try:
                    resp = json.loads(json_str)
                    
                    if resp["type"] == "content":
                        print(resp["content"], end="", flush=True)
                    elif resp["type"] == "tool_call":
                        print(f"\n[工具调用] {resp['content']['tool_name']}")
                    elif resp["type"] == "tool_result":
                        print(f"\n[工具返回结果]")
                    elif resp["type"] == "usage":
                        print(f"\n\n[Token使用统计] {resp['content']}")
                except:
                    pass
else:
    print(f"❌ 无法找到文件名 '{filename}' 对应的知识库ID")
    print("提示：请先运行上面的代码上传文件到RAG服务")


正在查找文件名 'test.pdf' 对应的知识库ID...
✅ 找到知识库ID: 2fb820d3-5cb9-46d5-9126-fa5b7412c36c

使用获取到的kb_id进行查询...
查询请求: 请查询知识库2fb820d3-5cb9-46d5-9126-fa5b7412c36c中关于Transformer的内容

我需要使用语义搜索工具来查询您指定的知识库。从您提供的信息来看，您希望查询知识库ID为"2fb820d3-5cb9-46d5-9126-fa5b7412c36c"中关于Transformer的内容。

让我为您进行查询：
[工具调用] semantic_search

[工具返回结果]
根据查询结果，我从知识库"2fb820d3-5cb9-46d5-9126-fa5b7412c36c"中找到了关于Transformer的详细内容。以下是主要信息：

## Transformer概述

Transformer是由论文《Attention is All You Need》提出的模型，现在是谷歌云TPU推荐的参考模型。它解决了传统RNN无法并行计算和长距离信息遗忘的问题，彻底摒弃了RNN和CNN结构，转而使用注意力机制。

## Transformer整体结构与工作流程

### 1. 整体结构
Transformer采用Encoder-Decoder结构：
- **Encoder（编码器）**：负责输入序列特征编码
- **Decoder（解码器）**：负责输出序列生成

### 2. 工作流程

#### 第一步：嵌入（Embedding）
包含两个子步骤：
1. **词嵌入（token-embedding）**：可以通过Word2Vec、One-hot、Glove等算法预训练得到，也可以在Transformer中训练得到
2. **位置编码（position-embedding）**：使用特定公式计算，使模型能够：
   - 处理比训练集更长的句子
   - 容易计算出相对位置

#### Encoder单层结构（TransformerBlock）
按顺序执行以下操作：
1. **多头自注意力层（MultiHeadAttention）**：捕捉输入序列内部token之间的依赖关系
2. **第一次Add & Norm**：残差连接 + 层归

In [5]:
# ========== 方式3: 传统方式 - 使用kb_id查询（对比） ==========
# 这是原来的方式，需要记住kb_id

rag_base_url = "http://localhost:5000"
base_url = "http://localhost:8000"
file_path = "./test.pdf"

def get_kb_id(rag_service_base_url: str, file_path: str, extract_images: bool = False): 
    """上传文件并返回kb_id"""
    try: 
        upload_url = f"{rag_service_base_url}/upload-pdf?extract_images={extract_images}"
        with open(file_path, "rb") as f: 
            upload_response = requests.post(upload_url, files={"file": f})
        
        if upload_response.status_code != 200:
            return {"kb_id": "NAN", "token_stats": 0}
        
        data = upload_response.json()["data"]
        return {"kb_id": data["knowledge_base_id"], "token_stats": data["token_stats"]}
    except Exception as e: 
        print(f"错误：{e}")
        return {"kb_id": "NAN", "token_stats": 0}

# 上传文件获取kb_id
response = get_kb_id(rag_service_base_url=rag_base_url, extract_images=False, file_path=file_path)
kb_id = response["kb_id"]

if kb_id != "NAN":
    print(f"文件解析成功! 知识库ID: {kb_id}")
    
    url = "http://localhost:8000/v1/chat/completions"
    FIXED_SESSION_ID = "my_test_session_01"
    
    request_data = {
        "user_id": "test_user3",
        "session_id": FIXED_SESSION_ID,
        "query": f"你好, 请介绍一下知识库{kb_id}的关于Transformer的内容"
    }
    
    response = requests.post(
        url,
        json=request_data,
        stream=True,
        headers={"Content-Type": "application/json"}
    )
    
    for line in response.iter_lines():
        if line:
            line_str = line.decode("utf-8").strip()
            if line_str.startswith("data: "):
                json_str = line_str[6:]
                try:
                    resp = json.loads(json_str)
                    
                    if resp["type"] == "content":
                        print(resp["content"], end="", flush=True)
                    elif resp["type"] == "tool_call":
                        print(resp)
                    elif resp["type"] == "tool_result":
                        print("工具返回结果: ", resp["content"])
                    elif resp["type"] == "usage":
                        print(resp)
                except:
                    pass
else:
    print("文件上传失败")


文件解析成功! 知识库ID: 8b70d6e6-4df0-4f61-bd88-bc461ed6ebb5
我看到您想了解知识库`8b70d6e6-4df0-4f61-bd88-bc461ed6ebb5`中关于Transformer的内容。让我为您调用语义搜索工具：{'type': 'tool_call', 'content': {'tool_name': 'semantic_search', 'tool_call_id': 'call_00_d3fAE5vYRZVNnUsZg5Fhfn8a', 'tool_args': {}}}
工具返回结果:  {'called_tool_name': 'semantic_search', 'called_tool_content': '{"content": "Transformer\\n基本背景\\n Transformer由论文《Attention is All You Need》提出，现在是谷歌云TPU推荐的参考模型。论文相关的\\nTensorflow的代码可以从GitHub获取，其作为Tensor2Tensor包的一部分。哈佛的NLP团队也实现了一个基于\\nPyTorch的版本，并注释该论文。\\n Transformer解决了RNN无法并行计算， 长距离信息遗忘的问题, 并且彻底摒弃RNN, CNN结构来计算，转而使用\\nAttention 注意力机制来解决问题。\\n1. Transformer 整体结构与工作流程\\n1.1 Transformer的整体结构\\n首先介绍Transformer的整体结构，传统的 Transformer是 Encoder-Decoder 结构。\\n下图是Transformer的总体结构 (重要) :\\n\\n可以看到 Transformer由Encoder(左)和Decoder(右)两个部分 组成。\\nEncoder（编码器）：输入序列特征编码\\n前置处理：词嵌入token-embedding 和 位置编码 position-embedding\\n单层结构（TransformerBlock）（注意: 顺序执行）：\\n多头自注意力(MultiHeadAttention) 层（捕捉输入序列内部 token 依赖）\\n第一次 Add & Norm（残差连接 + 层归一化,